# Importing Trained Model

In [93]:
from tensorflow.keras.models import load_model
# Load the model
model = load_model('/content/best_model.h5')

# Predictiing emotion based on Image

Run the next two code blocks for Google Colab

In [88]:
# Pre-processing image
from tensorflow.keras.preprocessing import image
import numpy as np
import cv2

# Path to the image file
img_path = 'sad.jpg'

In [96]:
def predict_emotion(img_path, model):
  # Load and preprocess the image
  img = image.load_img(img_path, target_size=(48, 48), color_mode='grayscale')
  img_array = image.img_to_array(img)
  img_array = np.expand_dims(img_array, axis=0)
  img_array /= 255.0  # Normalize pixel values to between 0 and 1

  predictions = model.predict(img_array)

  predicted_label_index = np.argmax(predictions)

  # {0 : 'Angry', 1 : 'Fear', 2 : 'Happiness', 3 : 'Sad', 4 : 'Neutral'}
  emotion_mapping = ['Angry', 'Fear', 'Happy', 'Sad', 'Neutral']
  predicted_emotion = emotion_mapping[predicted_label_index]
  return predicted_emotion

In [97]:
print(f'The predicted emotion for the image is: {predict_emotion(img_path, model)}')

1/1 [==============================] - 0s 30ms/step
The predicted emotion for the image is: Happy


# Predicting emotion with camera

In [27]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))

  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [86]:
predict_emotion('photo.jpg', model)

1/1 [==============================] - 0s 20ms/step


'Sad'